In [1]:
from sympy.physics.quantum.spin import Rotation
from numpy import *
from sympy import pi, symbols
import sympy
from sympy import I, sin, cos, exp
import transformations as tra

In [2]:
def f_App_A2(l1,l2,m,ri,rj):
    f = (-1)**(m+l2) \
    *math.factorial(l1+l2) \
    *(math.factorial(l1+abs(m)) \
     *math.factorial(l1-abs(m)) \
     *math.factorial(l2+abs(m)) \
     *math.factorial(l2-abs(m)) )**(-.5) \
    *ri**l1 *rj**l2
    return f

def my_euler_matrix(omega):
    alpha = omega[0]
    beta = omega[1]
    gamma = omega[2]
    Re = tra.euler_matrix(alpha,beta,gamma, 'rxyz')
    return matrix(Re)

def my_euler_from_matrix(Re):
    al, be, ga = tra.euler_from_matrix(Re, 'rxyz')
    omega = [al,be,ga]
    return omega

# omega = [0, pi/2, 0]
# Re = my_euler_matrix(omega)
# newomega = my_euler_from_matrix(Re)
# print newomega

In [3]:
def expand_in_R(l1max,l2max,ri,rj,omega1i_bar,omega2j_bar,R):
    brackets = 0
    for l1 in range(l1max+1):
        for l2 in range(l2max+1):
            lmin = min(l1,l2)
            for m in range (-lmin,lmin+1):
                D1i = Rotation.D(l1, m, 0,omega1i_bar[0], omega1i_bar[1],0).doit()
                D2j = Rotation.D(l2, m, 0,omega2j_bar[0], omega2j_bar[1],0).doit()
                f = f_App_A2(l1,l2,m,ri,rj)
                brackets += D1i*conj(D2j)*f/R**(l1+l2)
    rij = eval(str(R / brackets))
    try:
        rij = rij.expand(complex=True)
    except:
        pass
    rij = eval(str(rij))
    return rij, brackets


In [30]:
def expand_in_Rp(l1max,l2max,ri,rj,omega1i_bar,omega2j_bar,R):
    brackets = 0
    for l1 in range(l1max+1):
        for l2 in range(l2max+1):
            lmin = min(l1,l2)
            for m in range (-lmin,lmin+1):
                D1i = Rotation.D(l1, m, 0,omega1i_bar[0], omega1i_bar[1], omega1i_bar[2]).doit()
                D2j = Rotation.D(l2, m, 0,omega2j_bar[0], omega2j_bar[1], omega2j_bar[2]).doit()
                f = f_App_A2(l1,l2,m,ri,rj)
                brackets += D1i*conj(D2j)*f/R**(l1+l2)
    rij = eval(str(R / brackets))
    try:
        rij = rij.expand(complex=True)
    except:
        pass
    rij = eval(str(rij))
    return rij, brackets


In [4]:
def expand_in_omega_R(l1max,l2max,ri,rj,omega1i_bar,omega2j_bar,omega1,omega2,R):
    brackets = 0
    for l1 in range(l1max+1):
        for l2 in range(l2max+1):
            lmin = min(l1,l2)
            for m in range (-lmin,lmin+1):
                D1i = 0
                for mp in range(-l1,l1+1):
                    temp1 = Rotation.D(l1, m, mp, omega1[0], omega1[1], omega1[2]).doit()
                    temp2 = Rotation.D(l1, mp, 0, omega1i_bar[0], omega1i_bar[1], omega1i_bar[2]).doit()
                    D1i += temp1*temp2
                D2j = 0
                for mp in range(-l2,l2+1):
                    temp1 = Rotation.D(l2, m, mp, omega2[0], omega2[1], omega2[2]).doit()
                    temp2 = Rotation.D(l2, mp, 0, omega2j_bar[0], omega2j_bar[1], omega2j_bar[2]).doit()
                    D2j += temp1*temp2
                f = f_App_A2(l1,l2,m,ri,rj)
                brackets += D1i*conj(D2j)*f/R**(l1+l2)
    rij = eval(str(R / brackets))
    try:
        rij = rij.expand(complex=True)
        brackets = brackets.expand(complex=True)
    except:
        pass
    return rij, brackets

# These are handy ways to force a numerical evaluation ...
# rij = eval(str(rij))
# rij = rij.subs(pi, 3.1415926535)

In [5]:
def expand_in_omega_R_psi(l1max,l2max,ri,rj,omega1i_bar,omega2j_bar,omega1,omega2,R,psi):
    brackets = 0
    for l1 in range(l1max+1):
        for l2 in range(l2max+1):
            lmin = min(l1,l2)
            for m in range (-lmin,lmin+1):
                D1i = 0
                for mp in range(-l1,l1+1):
                    temp1 = 0
                    for mpp in range(-l1,l1+1):
                        tempp1 = Rotation.D(l1, m, mpp,  0,psi,0).doit()
                        tempp2 = Rotation.D(l1, mpp, mp, omega1[0], omega1[1], omega1[2]).doit()
                        temp1 += tempp1*tempp2
                    temp2 = Rotation.D(l1, mp, 0, omega1i_bar[0], omega1i_bar[1], omega1i_bar[2]).doit()
                    D1i += temp1*temp2
                D2j = 0
                for mp in range(-l2,l2+1):
                    temp1 = 0
                    for mpp in range(-l2,l2+1):
                        tempp1 = Rotation.D(l2, m, mpp,  0,psi,0).doit()
                        tempp2 = Rotation.D(l2, mpp, mp, omega2[0], omega2[1], omega2[2]).doit()
                        temp1 += tempp1*tempp2
                    temp2 = Rotation.D(l2, mp, 0, omega2j_bar[0], omega2j_bar[1], omega2j_bar[2]).doit()
                    D2j += temp1*temp2
                f = f_App_A2(l1,l2,m,ri,rj)
                brackets += D1i*conj(D2j)*f/R**(l1+l2)
    rij = eval(str(R / brackets))
    try:
        rij = rij.expand(complex=True)
        brackets = brackets.expand(complex=True)
    except:
        pass
    return rij, brackets

In [6]:
def evaluateRomegas(rij_sym, \
               R_sym, R_num, \
               omega1_sym, omega1_num, \
               omega2_sym, omega2_num \
             ):
    # Evaluate for numerical values of R, psi, etc
    alpha1_sym =  omega1_sym[0]; alpha1_num =  omega1_num[0]
    beta1_sym  =  omega1_sym[1]; beta1_num  =  omega1_num[1]
    gamma1_sym =  omega1_sym[2]; gamma1_num =  omega1_num[2]
    alpha2_sym =  omega2_sym[0]; alpha2_num =  omega2_num[0]
    beta2_sym  =  omega2_sym[1]; beta2_num  =  omega2_num[1]
    gamma2_sym =  omega2_sym[2]; gamma2_num =  omega2_num[2] 
    rij = rij_sym.subs([ \
                  (R_sym,R_num),\
                  (alpha1_sym,alpha1_num),\
                  (beta1_sym,beta1_num),\
                  (gamma1_sym,gamma1_num),\
                  (alpha2_sym,alpha2_num),\
                  (beta2_sym,beta2_num),\
                  (gamma2_sym,gamma1_num),\
               ]
               )
    rij = eval(str(rij))
    return rij

def evaluatepsi(rij_sym, \
               psi_sym, psi_num):

    rij = rij_sym.subs(psi_sym,psi_num)
    return rij

In [21]:
def showbrackets(brackets_sym):

    # This is to look at the expansion of the term in brackets
    alpha1, beta1, gamma1 = symbols('alpha1 beta1 gamma1', real=True)
    alpha2, beta2, gamma2 = symbols('alpha2 beta2 gamma2', real=True)
    R, psi = symbols('R psi', real=True)

    # These substitutions are just for aesthetics
    brackets_sym = brackets_sym.subs([ \
                  (R_sym,R),\
                  (alpha1_sym,alpha1),\
                  (beta1_sym,beta1),\
                  (gamma1_sym,gamma1),\
                  (alpha2_sym,alpha2),\
                  (beta2_sym,beta2),\
                  (gamma2_sym,gamma1),\
                 ])

    try:
        brackets_sym = brackets_sym.subs(psi_sym,psi)
    except:
        pass

    # Print the term in brackets
    #sympy.pretty_print(brackets_sym)

    # Print terms according to powers of 1/R
    x = symbols('x', real=True)
    test = brackets_sym.subs(R, 1/x)
    for i in range(4):
        term = test.diff(x,i).subs(x,0)/math.factorial(i)
        print "Coefficient of 1/R^", i
        #sympy.pretty_print(term)
        print(term)
        print "\n"

In [29]:
for m in range(3):
    print Rotation.D(2, m, 0, 0,0,0).doit()

1
0
0


In [8]:
# Specify the order of the expansion
l1max = 2
l2max = 1
small = 1e-3

# This is the distance between frames
R_num = 10

In [9]:
# This is the original formulation

# Specify the atomic configurations
beta1i = 1.5;      alpha1i = 0.1;  ri = 1; omega1i_bar = [alpha1i,beta1i,0]; 
beta2j = 0;        alpha2j = 0;  rj = 1; omega2j_bar = [alpha2j,beta2j,0]; 

# Expand
print omega1i_bar
print omega2j_bar
rij, brackets = expand_in_R(l1max,l2max,ri,rj,omega1i_bar,omega2j_bar,R_num)
print rij

[0.1, 1.5, 0]
[0, 0, 0]
11.083875375


In [32]:
# Here, start with atoms along the z-axis, and rotate the frames to match the above

# Specify the atomic configurations
beta1i = 0;     alpha1i = 0;  ri = 1; omega1i_bar = [alpha1i,beta1i,0]; 
beta2j = 0;     alpha2j = 0;  rj = 1; omega2j_bar = [alpha2j,beta2j,0]; 

# Specify a frame rotation for each atom
alpha1_num = 0.1
beta1_num = 1.5
gamma1_num = 0
omega1_num = [alpha1_num,beta1_num,gamma1_num]

alpha2_num = 0
beta2_num =  0
gamma2_num = 0
omega2_num = [alpha2_num,beta2_num,gamma2_num]

# Translate to rotation matrices, combine them, and get back the euler angles
R_omega1i_bar = my_euler_matrix(omega1i_bar)
R_omega2j_bar = my_euler_matrix(omega2j_bar)
R_omega1 = my_euler_matrix(omega1_num)
R_omega2 = my_euler_matrix(omega2_num)
R_omega1_omega1i_bar = R_omega1*R_omega1i_bar
R_omega2_omega2j_bar = R_omega2*R_omega2j_bar
omega1i_bar_p = my_euler_from_matrix(R_omega1_omega1i_bar)
omega2j_bar_p = my_euler_from_matrix(R_omega2_omega2j_bar)

# Expand using these new euler angles (which should be the same as the previous ones)
print omega1i_bar_p
print omega2j_bar_p
rij, brackets = expand_in_R(l1max,l2max,ri,rj,omega1i_bar_p,omega2j_bar_p,R_num)
print rij

[0.1, 1.5, -0.0]
[-0.0, 0.0, -0.0]
11.083875375


In [33]:
# Now try an omega-R expansion of the Wigner D-matrices instead
rij, brackets = expand_in_omega_R(l1max,l2max,ri,rj,omega1i_bar,omega2j_bar,omega1_num,omega2_num,R_num)
print rij

11.083875375


In [34]:
# Now try an R-expansion of the Wigner D-matrices alone
rij, brackets = expand_in_Rp(l1max,l2max,ri,rj,omega1_num,omega2_num,R_num)
print rij

11.083875375


In [35]:
# Now try an expansion of the Wigner D-matrices, but with symbolic variables

# Create the symbolic values
R_sym = symbols('R_sym', real=True)
alpha1_sym, beta1_sym, gamma1_sym = symbols('alpha1_sym beta1_sym gamma1_sym', real=True)
alpha2_sym, beta2_sym, gamma2_sym = symbols('alpha2_sym beta2_sym gamma2_sym', real=True)
omega1_sym = [alpha1_sym, beta1_sym, gamma1_sym];
omega2_sym = [alpha2_sym, beta2_sym, gamma2_sym]; 

# Expand
rij_sym, brackets_sym = expand_in_omega_R(l1max,l2max,ri,rj,omega1i_bar,omega2j_bar,omega1_sym,omega2_sym,R_sym)
#print rij_sym

In [36]:
# To check, evaluate the symbolic variables at numerical values 
rij_eval = evaluateRomegas(rij_sym, \
               R_sym, R_num, \
               omega1_sym, omega1_num, \
               omega2_sym, omega2_num \
             )
print rij_eval

11.0746691061


In [26]:
# Also good to check the brackets
showbrackets(brackets_sym)

Coefficient of 1/R^ 0
1.00000000000000


Coefficient of 1/R^ 1
1.0*cos(beta1) - 1.0*cos(beta2)


Coefficient of 1/R^ 2
sin(alpha1)*sin(alpha2)*sin(beta1)*sin(beta2) + sin(beta1)*sin(beta2)*cos(alpha1)*cos(alpha2) - 2.0*cos(beta1)*cos(beta2) + 0.75*cos(2*beta1) + 0.25


Coefficient of 1/R^ 3
0.866025403784439*sqrt(3)*sin(alpha1)*sin(alpha2)*sin(2*beta1)*sin(beta2) + 0.866025403784439*sqrt(3)*sin(2*beta1)*sin(beta2)*cos(alpha1)*cos(alpha2) - 2.25*cos(2*beta1)*cos(beta2) - 0.75*cos(beta2)




In [17]:
# Now we try an expansion of the Wigner D-matrices, with angle psi applied to both

# This is the angle ... if psi=0, this result should equal the box above
psi_num = 0

# Expand 
rij, brackets = \
    expand_in_omega_R_psi(l1max,l2max,ri,rj,omega1i_bar,omega2j_bar,omega1_num,omega2_num,R_num,psi_num)
print rij

11.083875375


In [18]:
# Now we try the same with symbolic variables

# Create the symbolic variable
psi_sym = symbols('psi_sym', real=True)

# Expand
rij_sym, brackets_sym = \
    expand_in_omega_R_psi(l1max,l2max,ri,rj,omega1i_bar,omega2j_bar,omega1_sym,omega2_sym,R_sym,psi_sym)
#print rij_sym

In [19]:
# To check, evaluate the symbolic variables at numerical values 
rij_eval1 = evaluateRomegas(rij_sym, \
               R_sym, R_num, \
               omega1_sym, omega1_num, \
               omega2_sym, omega2_num \
             )
rij_eval2 = evaluatepsi(rij_eval1, psi_sym, psi_num)
print rij_eval2

11.0894142940430


In [22]:
# Also good to check the brackets
showbrackets(brackets_sym)

Coefficient of 1/R^ 0
1.00000000000000


Coefficient of 1/R^ 1
-1.0*sin(beta1)*sin(psi)*cos(alpha1) + 1.0*sin(beta2)*sin(psi)*cos(alpha2) + 1.0*cos(beta1)*cos(psi) - 1.0*cos(beta2)*cos(psi)


Coefficient of 1/R^ 2
sin(alpha1)*sin(alpha2)*sin(beta1)*sin(beta2) + 0.75*sin(beta1)**2*sin(psi)**2*cos(2*alpha1) - 2.0*sin(beta1)*sin(beta2)*sin(psi)**2*cos(alpha1)*cos(alpha2) + sin(beta1)*sin(beta2)*cos(alpha1)*cos(alpha2)*cos(psi)**2 + 3.0*sin(beta1)*sin(psi)*cos(alpha1)*cos(beta2)*cos(psi) - 0.75*sin(2*beta1)*sin(2*psi)*cos(alpha1) + 3.0*sin(beta2)*sin(psi)*cos(alpha2)*cos(beta1)*cos(psi) + sin(psi)**2*cos(beta1)*cos(beta2) - 2.0*cos(beta1)*cos(beta2)*cos(psi)**2 + 0.5625*cos(2*beta1)*cos(2*psi) + 0.1875*cos(2*beta1) + 0.1875*cos(2*psi) + 0.0625


Coefficient of 1/R^ 3
0.866025403784439*sqrt(3)*sin(alpha1)*sin(alpha2)*sin(2*beta1)*sin(beta2)*cos(psi) - 0.21650635094611*sqrt(3)*sin(2*alpha1)*sin(alpha2)*sin(beta1)**2*sin(beta2)*sin(psi)*cos(psi) - 0.866025403784439*sqrt(3)*sin(2*alpha1)*sin(a